In [14]:
import pywren
import pickle 
import time
import numpy as np

def latency(data):
    from redis import Redis
    n = data[0]
    id = data[1]
    
    gateway_ip = '54.212.247.168'
    gateway_port = '8888'
    bind_port = '4000'+str(id)
    redis = Redis(gateway_ip, int(gateway_port), bind_port=int(bind_port))
    
    random_num = np.random.rand(1, n) #generate n floats (each becomes 20B after serializing)
    rand_serial = pickle.dumps(random_num)
    t0 = time.time()
    for i in range(100):
        redis.set(i, rand_serial)
    t1 = time.time()
    for i in range(100):
        redis.get(i)
    t2 = time.time()
    return (n, (t1-t0)*10, (t2-t1)*10) # time in ms 

In [33]:
def latency_no_gateway(data):
    from redis import Redis
    n = data[0]
    id = data[1]
    
    gateway_ip = '54.212.247.168'
    gateway_port = '8888'
    n_digits = id/10+1
    if n_digits == 1:
        bind_port = '4000'+str(id)
    elif n_digits == 2:
        bind_port = '400'+str(id)
    elif n_digits == 3:
        bin_port = '40'+str(id)
    redis = Redis(gateway_ip, int(gateway_port), bind_port=int(bind_port))
    t3 = time.time()
    redis.set('k', 'v') # excluding the gateway roundtrip time 
    t4 = time.time()
    
    random_num = np.random.rand(1, n) #generate n floats (each becomes 20B after serializing)
    rand_serial = pickle.dumps(random_num)
    t0 = time.time()
    for i in range(100):
        redis.set(i, rand_serial)
    t1 = time.time()
    for i in range(100):
        redis.get(i)
    t2 = time.time()
    return (n, (t1-t0)*10, (t2-t1)*10, (t4-t3)*1000) # time in ms 

In [16]:
wrenexec = pywren.default_executor()

In [24]:
args_list = []
for i in range(1):
    args_list.append([1, i])

time0 = time.time()
futures = wrenexec.map(latency_no_gateway, args_list)
results = pywren.get_all_results(futures)
time1 = time.time()
t_total = time1-time0
print str(t_total) + " sec"

result = [0, 0, 0]
result_full = [[], [], []]
for res in results:
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
    result[2] += res[3]
    result_full[2].append(res[3])
result[0] /= len(futures)
result[1] /= len(futures)
result[2] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
result_gateway = np.array(result_full[2])
print [np.percentile(result_write, 100), np.percentile(result_read, 100), np.percentile(result_gateway, 100)]
print result #avg write time, avg read time

7.4863011837 sec
[2.205958366394043, 2.2042608261108398, 1006.5159797668457]
[2.205958366394043, 2.20426082611084, 1006.5159797668457]


In [29]:
args_list = []
n_lambda = 10
print str(n_lambda) + " lambdas" 
for i in range(n_lambda):
    args_list.append([1, i])

time0 = time.time()
futures = wrenexec.map(latency_no_gateway, args_list)
results = pywren.get_all_results(futures)
time1 = time.time()
t_total = time1-time0
print str(t_total) + " sec"

result = [0, 0, 0]
result_full = [[], [], []]
for res in results:
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
    result[2] += res[3]
    result_full[2].append(res[3])
result[0] /= len(futures)
result[1] /= len(futures)
result[2] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
result_gateway = np.array(result_full[2])
print [np.percentile(result_write, 100), np.percentile(result_read, 100), np.percentile(result_gateway, 100)]
print result #avg write time, avg read time

10 lambdas
14.7844059467 sec
[32.402219772338867, 29.993259906768799, 1193.659782409668]
[28.87881326675415, 27.22055411338806, 889.5804643630981]


In [34]:
args_list = []
n_lambda = 100
print str(n_lambda) + " lambdas" 
print "test"
for i in range(n_lambda):
    args_list.append([1, i])

time0 = time.time()
futures = wrenexec.map(latency_no_gateway, args_list)
results = pywren.get_all_results(futures)
time1 = time.time()
t_total = time1-time0
print str(t_total) + " sec"

result = [0, 0, 0]
result_full = [[], [], []]
for res in results:
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
    result[2] += res[3]
    result_full[2].append(res[3])
result[0] /= len(futures)
result[1] /= len(futures)
result[2] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
result_gateway = np.array(result_full[2])
print [np.percentile(result_write, 100), np.percentile(result_read, 100), np.percentile(result_gateway, 100)]
print result #avg write time, avg read time

100 lambdas
test


UnboundLocalError: local variable 'bind_port' referenced before assignment

In [ ]:
args_list = []
n_lambda = 500
print str(n_lambda) + " lambdas" 
for i in range(n_lambda):
    args_list.append([1, i])

time0 = time.time()
futures = wrenexec.map(latency_no_gateway, args_list)
results = pywren.get_all_results(futures)
time1 = time.time()
t_total = time1-time0
print str(t_total) + " sec"

result = [0, 0, 0]
result_full = [[], [], []]
for res in results:
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
    result[2] += res[3]
    result_full[2].append(res[3])
result[0] /= len(futures)
result[1] /= len(futures)
result[2] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
result_gateway = np.array(result_full[2])
print [np.percentile(result_write, 100), np.percentile(result_read, 100), np.percentile(result_gateway, 100)]
print result #avg write time, avg read time